# STEP3 TA IndéMode  データ取得 近隣サロン

>### 環境構築

ターミナルにて  
 ● pip install requests  
 ● pip install beautifulsoup4  
 ● pip install gspread  
 ● pip install google-auth

>### 活用サイト

楽天Beauty  
https://beauty.rakuten.co.jp/

規約確認  
✔ https://beauty.rakuten.co.jp/robots.txt  
　→「Disallow:/rent_store 」を検索 → 無し  
✔ プライバシーポリシー  
　→スクレイピング・クローリング禁止の記載なし

>### コード

In [39]:
# ライブラリインポート
from time import sleep
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread 
from google.oauth2 import service_account
import json

# スプレッドシートの認証設定
SP_CREDENTIAL_FILE = 'gspread-test-399313-ca3a69c4d75e.json' #個人の認証キーのjsonファイルのパスを記入
SP_COPE = [
    'https://www.googleapis.com/auth/drive',
    'https://spreadsheets.google.com/feeds'
]
credentials = service_account.Credentials.from_service_account_file(SP_CREDENTIAL_FILE, scopes=SP_COPE)
gc = gspread.authorize(credentials)

#スプレッドシート/シート指定
SP_SHEET_KEY = '1474-fT6RkuGOvPwjyCJIEtD6qrNjI6EP2bw3vtHvLBc' #川名のスプレッドシート
SP_SHEET = '最寄駅近隣サロンDB'

# 情報格納のための空配列を用意
s_name = []
s_access = []
s_price = []


# HTML取得
URL = 'https://beauty.rakuten.co.jp/eki571/page{}/'

# 複数ページ取得のためのループ処理
for i in range(1,6):

    # 変数trget_urlに、アクセス先のURLを格納する
    target_url = URL.format(i)

    #サーバー負荷低減のための1s待機
    sleep(1)

    #print()してtrget_url確認
    print(target_url)

    # リクエスト
    res = requests.get(target_url)

    # リクエスト結果を確認
    #result = res.text
    #print(result)

    # 文字コード変換 (→ utf-8 )
    res.encoding = 'utf-8'

    # 文字化けしていないかhtmlを確認
    #print(res.text)

    # BeautifulSoup(解析したいデータ,解析する方法)を指定し、soupに代入
    soup = BeautifulSoup(res.text, "html.parser")
    #print(soup)

    # sectionデータ(1物件分データ)の取得
    property_section = soup.select('div.m-shopCard__headerContent')

    # 取得したセクションすべての数を確認します。
    print(len(property_section))

    # property_sectionからsectionを1つずつ取り出してpsに代入。
    for ps in property_section:
        ps1 = ps.select('h3.m-shopCard__shopName')[0].text
        ps2 = ps.select('li.m-shopCard__headerOutlineItem')[0].text.replace("アクセス：","")
        ps3 = ps.select('li.m-shopCard__headerOutlineItem')[1].text.replace("カット単価：","").replace("～","").replace("￥","")

        # それぞれ抽出したデータを配列に追加
        s_name.append(ps1)
        s_access.append(ps2)
        s_price.append(ps3)
        
    # 物件数確認
    #print('サロン件数', len(s_name))

# データ定義
data_list = {
    "サロン名" : s_name,
    "アクセス" : s_access,
    "カット単価(円~)" : s_price,
}

# データフレーム作成
df = pd.DataFrame(data_list)

# 重複データ処理
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)
#display(df)

# csv保存
#df.to_csv('東京都新宿区の貸店舗・テナント物件一覧_店舗ネットワーク.csv', index=False)

# スプレッドシート/シートを選択して初期化既存データ削除）
sh = gc.open_by_key(SP_SHEET_KEY)
worksheet = sh.worksheet(SP_SHEET)
worksheet.clear() 

# DataFrameをシートに書き込む
#worksheet.update([filtered_df.columns.values.tolist()] + filtered_df.values.tolist())
worksheet.update(values=[df.columns.values.tolist()] + df.values.tolist(), range_name='A1')

print('スプレッドシートへの書き込みが完了しました')

https://beauty.rakuten.co.jp/eki571/page1/
30
https://beauty.rakuten.co.jp/eki571/page2/
30
https://beauty.rakuten.co.jp/eki571/page3/
30
https://beauty.rakuten.co.jp/eki571/page4/
21
https://beauty.rakuten.co.jp/eki571/page5/
0
スプレッドシートへの書き込みが完了しました


In [33]:
ps = property_section[0]
ps1 = ps.select('h3.m-shopCard__shopName')[0].text
print(ps1)
ps2 = ps.select('li.m-shopCard__headerOutlineItem')[0].text
print(ps2)
ps3 = ps.select('li.m-shopCard__headerOutlineItem')[1].text
ps3

Euphoria GINZA GRANDE 銀座(ユーフォリア　ギンザ　グランデ)
アクセス：銀座駅 徒歩1分
、有楽町駅 徒歩5分


'カット単価：￥5,500～'

In [35]:
# データ確認
filtered_df

,物件名,使用階/部屋番号,所在地,最寄駅,床面積(m2),賃料(万円),保証金(万円),礼金(万円)
0,ローカルステイ中井,1/1階部分,東京都新宿区上落合２丁目19-13,都営大江戸線 中井,12.83,13.2,0,12
1,ＲＥＳＴＡ ＴＡＫＡＤＡＮＯＢＡＢＡ,-,東京都新宿区高田馬場3-2,ＪＲ山手線 高田馬場,125.05,195.47,700,0
2,高田馬場 第９０東京ビル ３階,3,東京都新宿区高田馬場3-5-3,ＪＲ山手線 高田馬場,191.22,88.0,600,0
3,西早稲田１階店舗,1,東京都新宿区西早稲田1-23-13,都電荒川線 早稲田,80.44,41.8,250,0
4,ＫＭプラザ ３Ｆ,3/302-303,東京都新宿区大久保１丁目16-27,ＪＲ山手線 新大久保,57.82,18.7,37,0
5,小玉ビル,4/403,東京都新宿区西新宿7丁目19-11,東京メトロ丸ノ内線 西新宿,20.91,11.0,0,12
6,ＭＡフラットビル,1,東京都新宿区高田馬場2-7-13,ＪＲ山手線 高田馬場,48.00,26.18,261,57
7,第二サンパークビル １階,1,東京都新宿区新宿3-21-4,ＪＲ山手線 新宿,59.10,181.5,1815,181
8,四谷林ビル,1棟/1階~3階,東京都新宿区四谷一丁目18-11,東京メトロ丸ノ内線 四ツ谷,256.15,132.0,792,0
9,恩田セントラルビル ４階,4/401,東京都新宿区新宿3-3-3,東京メトロ副都心線 新宿三丁目,62.35,62.24,622,68
